In [5]:
import sys
sys.path.append('..')

from random import randint

from lightfm import LightFM
import pandas as pd
import numpy as np
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import LightFMWrapperModel

from service.models.ann import ApproximateNearestNeighbors

# LOAD DATA

In [ ]:
!mkdir ../data
!wget https://storage.yandexcloud.net/itmo-recsys-public-data/kion_train.zip -O ../data/data_original.zip
!unzip ../data/data_original.zip -d ../data

In [6]:
interactions = pd.read_csv('../data/kion_train/interactions.csv')
users = pd.read_csv('../data/kion_train/users.csv')
items = pd.read_csv('../data/kion_train/items.csv')

# Preprocess

In [7]:
Columns.Datetime = 'last_watch_dt'

In [8]:
interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index, inplace=True)

In [9]:
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')

In [10]:
max_date = interactions[Columns.Datetime].max()

In [11]:
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)

In [12]:
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()

print(f"train: {train.shape}")
print(f"test: {test.shape}")

train: (4985269, 6)
test: (490982, 6)


In [13]:
train.drop(train.query("total_dur < 300").index, inplace=True)

In [14]:
# отфильтруем холодных пользователей из теста
cold_users = set(test[Columns.User]) - set(train[Columns.User])

In [15]:
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

# Models

In [16]:
dataset = Dataset.construct(interactions_df=train)

In [17]:
TEST_USERS = test[Columns.User].unique()

# Добавляем аватаров в обучающую выборку

1. Фанат фильмов с Киану Ривзом, смотрит фильмы и рекламу, только с его участием!
2. Человек-Патриот, смотрит фильмы только made in Russia. Считает что раньше было лучше!
3. Девочка, смотрит только мультфильмы про принцесс и ждёт своего принца на белом коне.

In [18]:
kianu_fanboy = items[(items.actors.isna() == False) & (items.actors.str.contains('Киану Ривз'))][:10]
patriot = items[(items.countries.isna() == False) & (items.countries.str.contains('Россия|СССР'))][:10]
princesses = items[(items.age_rating <= 12) & (items.keywords.str.contains('принцесса|королева', case=False))][:30]

new_items = [kianu_fanboy, patriot, princesses]

In [19]:
train_2 = train.copy()
max_user_id = train_2.user_id.max()
avatar_ids = list()

for user_items in new_items:
    max_user_id += 1
    for item_id in user_items.item_id:
        last_watch_dt = f'2022-{str(randint(1,12)).zfill(2)}-{str(randint(1,28)).zfill(2)}'
        if max_user_id not in avatar_ids:
            avatar_ids.append(max_user_id)
        train_2 = train_2.append({
            'user_id': max_user_id,
            'item_id': item_id,
            'last_watch_dt': last_watch_dt,
            'watched_pct': randint(70, 100),
            'weight': 3
        }, ignore_index=True)

/tmp/ipykernel_761384/1073467818.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_2 = train_2.append({
/tmp/ipykernel_761384/1073467818.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_2 = train_2.append({
/tmp/ipykernel_761384/1073467818.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_2 = train_2.append({
/tmp/ipykernel_761384/1073467818.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_2 = train_2.append({
/tmp/ipykernel_761384/1073467818.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_2 = train_2

In [20]:
avatar_map = pd.DataFrame({"user_id": avatar_ids, "name": ['kianu', 'patriot', 'pricesses']})

In [21]:
avatar_ids

[1097558, 1097559, 1097560]

# Обучение LightFm

In [38]:
dataset_2 = Dataset.construct(interactions_df=train_2)

/home/justyell/anaconda3/envs/recsys/lib/python3.8/site-packages/rectools/dataset/interactions.py:62: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df[Columns.Datetime] = df[Columns.Datetime].astype("datetime64[ns]")


In [39]:
model = LightFMWrapperModel(
        LightFM(
            no_components=12,
            loss='warp',
            random_state=1234,
        ),
        epochs=10,
        num_threads=4,
    )

model.fit(dataset_2)

In [51]:
recoms = model.recommend(
    users=avatar_ids,
    dataset=dataset_2,
    k=10,
    filter_viewed=True,
)
recoms = pd.merge(recoms, items, on='item_id')
recoms = pd.merge(recoms, avatar_map, on='user_id')[['name', 'countries', 'title', 'genres', 'age_rating', 'actors', 'keywords']]


# Approximate Nearest Neighbors

In [52]:
ann = ApproximateNearestNeighbors(model=model, dataset=dataset_2)
ann.fit(k_reco=10)

## Фанат Киану Ривза
Модель плохо справилась, в плане предсказания фильмов с данным актёром, однако в целом попадает в в жанры, в которых он снимался

In [53]:
lightfm_recs_kianu = recoms[:10]
ann_recs_kianu = items[items.item_id.isin(ann.predict(user_id=avatar_ids[0]))]

In [54]:
assert set(lightfm_recs_kianu.title) == set(ann_recs_kianu.title)

In [55]:
lightfm_recs_kianu

,name,countries,title,genres,age_rating,actors,keywords
0,kianu,Россия,Хрустальный,"триллеры, детективы",18.0,"Антон Васильев, Николай Шрайбер, Екатерина Оль...","хруст, хрусталь, хруста, хрус, полицейский, пе..."
1,kianu,"Великобритания, США",Гнев человеческий,"боевики, триллеры",18.0,"Джейсон Стэйтем, Холт МакКэллани, Джеффри Доно...","ограбление, криминальный авторитет, месть, пер..."
2,kianu,Россия,Девятаев,"драмы, военные, приключения",12.0,"Павел Прилучный, Павел Чинарёв, Тимофей Трибун...","Девятаев, Девятаева, Девят, Девя, Девята, Девя..."
3,kianu,Россия,Клиника счастья,"драмы, мелодрамы",18.0,"Дарья Мороз, Анатолий Белый, Данил Акутин, Мар...","Клиника счастья, Клиника, Счастье, Клиника сча..."
4,kianu,США,Мстители: Финал,"боевики, драмы, фантастика",16.0,"Роберт Дауни мл., Крис Эванс, Марк Руффало, Кр...","космическое путешествие, путешествие во времен..."
5,kianu,Россия,Секреты семейной жизни,комедии,18.0,"Петр Скворцов, Алена Михайлова, Федор Лавров, ...","брызги крови, кровь, жестокое обращение с живо..."
6,kianu,Россия,Прабабушка легкого поведения,комедии,16.0,"Александр Ревва, Глюкоза, Дмитрий Нагиев, Миха...",", 2021, россия, прабабушка, легкого, поведения"
7,kianu,Великобритания,Аферистка,"триллеры, комедии",18.0,"Розамунд Пайк, Питер Динклэйдж, Эйса Гонсалес,...","опекун, аферист, чёрная комедия, мошенник, 202..."
8,kianu,Россия,Маша,"драмы, триллеры",16.0,"Максим Суханов, Аня Чиповская, Полина Гухман, ...","Фильм Маша, Маша фильм 2021, Смотреть фильм Ма..."
9,kianu,США,Мстители: Война бесконечности,"боевики, фантастика, приключения",16.0,"Роберт Дауни мл., Крис Хемсворт, Марк Руффало,...","магия, жертва, супергерой, основанный на комик..."


## Человек-патриот
В рекомендациях 3/10 фильмов не из россии, не идеально, но приемлимо

In [56]:
lightfm_recs_patr = recoms[10:20]
ann_recs_patr = items[items.item_id.isin(ann.predict(user_id=avatar_ids[1]))]

In [57]:
assert set(lightfm_recs_patr.title) == set(ann_recs_patr.title)

In [58]:
lightfm_recs_patr

,name,countries,title,genres,age_rating,actors,keywords
10,patriot,Россия,Хрустальный,"триллеры, детективы",18.0,"Антон Васильев, Николай Шрайбер, Екатерина Оль...","хруст, хрусталь, хруста, хрус, полицейский, пе..."
11,patriot,"Великобритания, США",Гнев человеческий,"боевики, триллеры",18.0,"Джейсон Стэйтем, Холт МакКэллани, Джеффри Доно...","ограбление, криминальный авторитет, месть, пер..."
12,patriot,Россия,Девятаев,"драмы, военные, приключения",12.0,"Павел Прилучный, Павел Чинарёв, Тимофей Трибун...","Девятаев, Девятаева, Девят, Девя, Девята, Девя..."
13,patriot,Россия,Клиника счастья,"драмы, мелодрамы",18.0,"Дарья Мороз, Анатолий Белый, Данил Акутин, Мар...","Клиника счастья, Клиника, Счастье, Клиника сча..."
14,patriot,Россия,Секреты семейной жизни,комедии,18.0,"Петр Скворцов, Алена Михайлова, Федор Лавров, ...","брызги крови, кровь, жестокое обращение с живо..."
15,patriot,Россия,Прабабушка легкого поведения,комедии,16.0,"Александр Ревва, Глюкоза, Дмитрий Нагиев, Миха...",", 2021, россия, прабабушка, легкого, поведения"
16,patriot,Россия,Подслушано,"драмы, триллеры",16.0,"Александр Hовиков, Валентина Ляпина, Никита Па...","подслушано, подслушано в контакте, социальная ..."
17,patriot,Россия,Афера,комедии,18.0,"Сергей Степин, Игорь Царегородцев, Татьяна Лял...","Афера, Аферисты, Карантин, Пандемия, Карантин ..."
18,patriot,Австралия,2067: Петля времени,"драмы, фантастика",16.0,"Аарон Гленнэйн, Коди Смит-МакФи, Райан Квантен...","изменение климата, путешествие во времени, выр..."
19,patriot,Китай,Восемь сотен,"боевики, драмы, историческое, военные",18.0,"Ван Цяньюань, Цзян У, Хуан Чжичжун, Чжан Цзюнъ...","осада, китайско-японская война, 2020, китай, в..."


## Ребенок, смотрящий мультики про принцесс
В рекомендациях только мультики для детей, так что в "ребёнка" попали, однако из всего списка только 1 мультфильм про принцесс.

In [59]:
lightfm_recs_child = recoms[20:30]
ann_recs_child = items[items.item_id.isin(ann.predict(user_id=avatar_ids[2]))]

In [60]:
assert set(lightfm_recs_child.title) == set(ann_recs_child.title)

In [61]:
ann_recs_child

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
158,13915,film,Вперёд,Onward,2020.0,"для детей, приключения, семейное, фэнтези, ком...",США,NaN,6.0,NaN,Дэн Скэнлон,"Том Холланд, Крис Пратт, Джулия Луис-Дрейфус, ...","Когда-то давно в сказочном мире царила магия, ...","эльфы, мир фантазий, эльф, главный герой подро..."
1554,16270,film,Тайна Коко,Coco,2017.0,"мультфильм, фэнтези, приключения",США,NaN,12.0,NaN,"Ли Анкрич, Эдриан Молина","Энтони Гонсалес, Гаэль Гарсиа Берналь, Бенджам...",Мексиканский юноша Мигель живёт в семье сапожн...,"Мексика, гитара, музыкант, скелет, музыка, заг..."
3951,3182,film,Ральф против Интернета,Ralph Breaks the Internet,2018.0,"мультфильм, приключения, фантастика, семейное,...",США,NaN,6.0,NaN,"Рич Мур, Фил Джонстон","Джон Си Райли, Сара Силверман, Галь Гадот, Тар...",На этот раз Ральф и Ванилопа фон Кекс выйдут з...,"видеоигра, мультфильм, продолжение, интернет, ..."
5712,11985,film,История игрушек 4,Toy Story 4,2019.0,"мультфильм, фэнтези, комедии",США,NaN,6.0,NaN,Джош Кули,"Том Хэнкс, Тим Аллен, Энни Поттс, Тони Хейл, К...","Космический рейнджер Баз Лайтер, ковбой Вуди, ...","игрушка, дружба, ковбой, история игрушек 4, , ..."
9698,16166,film,Зверополис,Zootopia,2016.0,"приключения, мультфильм, детективы, комедии",США,NaN,6.0,NaN,"Байрон Ховард, Рич Мур, Джаред Буш","Джиннифер Гудвин, Джейсон Бейтман, Идрис Эльба...",Добро пожаловать в Зверополис — современный го...,"аллегория, лев, бегемот, лиса, слон, овца, бел..."
9921,10761,film,Моана,Moana,2016.0,"мультфильм, фэнтези, мюзиклы",США,NaN,6.0,NaN,"Рон Клементс, Джон Маскер, Дон Холл","Аулии Кравальо, Дуэйн Джонсон, Рэйчел Хаус, Те...","Бесстрашная Моана, дочь вождя маленького племе...","океан, парусник, море, мифология, остров, мюзи..."
9922,5023,film,101 далматинец,101 Dalmatians,1996.0,"семейное, для детей","США, Великобритания",NaN,6.0,NaN,Стивен Херек,"Гленн Клоуз, Джефф Дэниелс, Джоэли Ричардсон, ...",Эта забавная история произошла в Англии на Рож...,"лондон, англия, мех, модельер, щенок, шуба, по..."
12620,13243,film,Головоломка,Inside Out,2015.0,"фантастика, мультфильм, комедии",США,NaN,6.0,NaN,"Пит Доктер, Роналдо Дель Кармен","Эми Полер, Филлис Смит, Ричард Кайнд, Билл Хей...",11-летняя девочка Райли переезжает с семьёй из...,"мечта, мультфильм, воображаемый друг, начальна..."
14435,10876,film,Гарфилд,Garfield,2004.0,"мультфильм, фэнтези, комедии",США,NaN,0.0,NaN,Питер Хьюит,"Брекин Мейер, Дженнифер Лав Хьюитт, Стивен Тоб...","Хозяин кота Гарфилда, Джон, приносит в дом мил...","конкуренция, кошка, талисман, лазанья, домашни..."
15352,7582,film,Холодное сердце II,Frozen II,2019.0,"фэнтези, мультфильм, музыкальные",США,NaN,6.0,NaN,"Крис Бак, Дженнифер Ли","Идина Мензел, Кристен Белл, Джонатан Грофф, Дж...","Анна, Эльза, Кристоф, его верный олень Свен и ...","королева, магия, королевство, плотина, дух, же..."


ВЫВОД: Получили быстрее работающие предикты, без потери качества предсказаний